In [3]:
! pip install minio

  Using cached minio-7.2.7-py3-none-any.whl (93 kB)
  Using cached pycryptodome-3.20.0-cp35-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)


In [6]:
# import json
import time
import requests
import logging
# import boto3
# from botocore.client import Config
from minio import Minio
from confluent_kafka import Producer
from pyspark.sql import SparkSession

In [10]:
# client = Minio("s3.amazonaws.com")
# client = Minio("s3.amazonaws.com", "datalake", "datalake")
client = Minio(
    "play.minio.io:9000",
    access_key="dHDgY1EjEVQkZkpa",
    secret_key="23x2hxwhlsWpy6TqH4nSpXG3IJ86vSgS",
    
)
# client = Minio("play.min.io")


In [11]:
bucket_name = 's3a://raw/'

# Para listar objetos no bucket 'raw'
response = client.list_objects(bucket_name)
print(response)

<generator object Minio._list_objects at 0x7fb51d2b1d20>


In [ ]:
buckets = client.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

In [13]:

import json
import time
import requests
import logging
import os

from delta.tables import DeltaTable
from confluent_kafka import Producer
from pyspark.sql import SparkSession

logging.basicConfig(level=logging.INFO)

class ExtractDataApi:
    
    def __init__(self, url: str, request):
        
        self.req = request
        self.url = url        
    
    def get_data(self):
        
        try:
            response = self.req.get(self.url)
            if response.status_code == 200:
                return response.json()
            else:
                return None
        except Exception as e:
            raise e
        
class KafkaProducer:
    
    def __init__(self, producer:Producer):
        
        self.producer = producer({'bootstrap.servers': 'kafka-broker:9092'})
        
    def send_data(self, topic:str, data:dict):
        
        try:
            data = json.dumps(data)
            self.producer.produce(topic, value=data.encode('utf-8'), callback=self.verify)
            self.producer.flush()
            
        except Exception as e:
            raise e
    
    def verify(self, err, msg):
        if err is not None:
            logging.error(f'Message delivery failed: {err}')
        else:
            logging.info(f'Message delivered to {msg.topic} [{msg.partition()}], {msg.offset()}')

class SparkStreaming:
    
    def __init__(self):

        # self.spark = SparkSession.builder \
        #     .appName('ContabilizeiTeste') \
        #     .getOrCreate()
        self.spark = SparkSession.builder \
            .appName("MinIO Example") \
            .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9050") \
            .config("spark.hadoop.fs.s3a.access.key", "datalake") \
            .config("spark.hadoop.fs.s3a.secret.key", "datalake") \
            .config("spark.hadoop.fs.s3a.path.style.access", True) \
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", False) \
            .getOrCreate()
    #     self.sc = self.spark.sparkContext
    #     self._config_minio()
    
    # def _config_minio(self):
    #     try:
    #         self.sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", "datalake")
    #         self.sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "datalake")
    #         self.sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9050")
    #         self.sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
    #         self.sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
    #         self.sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    #         self.sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
        
        # except Exception as e:
        #     raise e
        
    def read_data_streaming(self, topic:str):
        
        df = self.spark.readStream \
            .format('kafka') \
            .option('kafka.bootstrap.servers', 'kafka-broker:9092') \
            .option('subscribe', topic) \
            .load()
        
        return df

    def write_data_streaming(self, df, output_path:str):
            
        query = df.writeStream \
            .format('json') \
            .outputMode('append') \
            .option('path', output_path) \
            .option('checkpointLocation', 'checkpoint') \
            .trigger(processingTime='30 seconds') \
            .start()
        
        return query.awaitTermination()

class DeltaOperations(SparkStreaming):

    def __init__(self):

        self.spark = super().spark
        self.delta_table = DeltaTable.forPath(self.spark, 'path')        

    def merge_data(self, df):
            
        commit = (
            self.delta_table
                .alias('old') 
                .merge(
                    df.alias('new'), 
                    'old.id = new.id'
                ) 
                .whenMatchedUpdateAll(
                  set={
                    "first_name": "new.first_name",
                    "last_name": "new.last_name",
                    "email": "new.email",
                    "date_of_birth": "new.date_of_birth"
                }) 
                .whenNotMatchedInsertAll(
                    values={
                    "id": "new.id",
                    "first_name": "new.first_name",
                    "last_name": "new.last_name",
                    "email": "new.email",
                    "date_of_birth": "new.date_of_birth"
                }
            ) 
            .execute()
        )
        return commit            
        
        



In [28]:
def run():

    url = 'https://random-data-api.com/api/v2/users?size=1'
    r = requests
    
    while True:
        
        print("Inicío da extração dos dados da API")
        e = ExtractDataApi(url, r)
        time.sleep(1)
        data = e.get_data()
        
        print("Iniciou a etapa Kafka")
        p = Producer
        queue = KafkaProducer(p)
        queue.send_data('user', data)
        print(data)
        
        time.sleep(2)
        print("Iniciou a etapa do Spark")
        spark = SparkStreaming()
        topic = 'user'
        df = spark.read_data_streaming(topic)
        
        print("Iniciou a gravação no S3")
        output_path = "s3a://raw/users/"
        spark.write_data_streaming(df, output_path)
        
#         time.sleep(3)
#         query = (
#             df.writeStream 
#               .queryName("my_streaming_table")  
#               .outputMode("append") 
#               .format("memory") 
#               .start()
#         )

#         spark.sql("SELECT * FROM my_streaming_table").show()

#         query.stop()

In [ ]:
run()

Inicío da extração dos dados da API


INFO:root:Message delivered to <built-in method topic of cimpl.Message object at 0x7fb50a583040> [0], 13


Iniciou a etapa Kafka
{'id': 8934, 'uid': '7b9130d9-5a3f-48c4-9714-1d0917d10d3c', 'password': 'C36adYTGSw', 'first_name': 'Alva', 'last_name': 'Kautzer', 'username': 'alva.kautzer', 'email': 'alva.kautzer@email.com', 'avatar': 'https://robohash.org/quiaoccaecativoluptatem.png?size=300x300&set=set1', 'gender': 'Agender', 'phone_number': '+229 718-436-2636 x940', 'social_insurance_number': '778245308', 'date_of_birth': '1998-11-28', 'employment': {'title': 'Future Government Representative', 'key_skill': 'Organisation'}, 'address': {'city': 'West Hisako', 'street_name': 'Mraz Stream', 'street_address': '78153 Jennifer Ford', 'zip_code': '08859-1522', 'state': 'Connecticut', 'country': 'United States', 'coordinates': {'lat': 51.121701497848875, 'lng': 52.398825338796655}}, 'credit_card': {'cc_number': '4495101785945'}, 'subscription': {'plan': 'Bronze', 'status': 'Active', 'payment_method': 'Google Pay', 'term': 'Full subscription'}}
Iniciou a etapa do Spark
Iniciou a gravação no S3
